In [ ]:
import pandas as pd
import time
import tweepy
from selenium.webdriver import Chrome

Querying our table for all reviewers who have written over 100 reviews. 

In [ ]:
query = """
SELECT
    author
,   COUNT(author) as reviews
FROM
    reviews
GROUP BY
    author;"""

cur.execute(query)
authors = [name[0] for name in cur.fetchall() if name[1] > 100]
len(authors)

Using Selenium to scrape as follows:
* Searching Pitchfork by reviewer name
* If they have a profile, go to that page
* If they have a Twitter account listed, grabbing the link, and returning it to a dictionary.

In [ ]:
author_twitter = []
baddys = []
browser = Chrome()

for name in authors:
    try:
        url_name = name.replace(" ","%20")
        url = "https://pitchfork.com/search/?query=" + url_name
        browser.get(url)
        time.sleep(5)
        page = browser.find_element_by_partial_link_text(name.title())
        browser.get(page.get_attribute('href'))
        social_links = [a.get_attribute('href') for a 
                        in browser.find_elements_by_class_name('author-socials__link') 
                        if "twitter" in a.get_attribute('href')]
        author_dict = {'author': name, 'twitter': social_links[0].replace('http://twitter.com/','') }
        author_twitter.append(author_dict)
        time.sleep(5)
    except Exception:
        baddys.append(name)

Accessing Twitter API via Tweepy Python package:

Finding the reviewers follower count and adding to the dictionary with their details.

In [ ]:
keys = []
with open('.secret/twitter.json') as f:
    for code in f:
        keys.append(code)

In [ ]:
auth = tweepy.OAuthHandler(code[0], code[1])
auth.set_access_token(code[2], code[3])

api = tweepy.API(auth)

In [ ]:
for author in author_twitter:
    user = api.get_user(author['twitter'])
    author['followers'] = user.followers_count
    a_twitter.append(author)

In [ ]:
twitter_df = pd.DataFrame(author_twitter)
twitter_df.rename(columns={'name': 'author'}, inplace=True)
twitter_df

Exporting the data as a CSV, for later import to PostGreSql:

In [ ]:
twitter_df.to_csv('author_twitter.csv')